VASP analysis by Peter H. Jacobse 2025

In [591]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection, LineCollection
import open3d as o3d
from pymatgen.core import Lattice, Structure
from pymatgen.electronic_structure.core import Spin
import pymatgen.io.vasp.outputs as vasp_out
from scipy.ndimage import zoom
from scipy.signal import convolve2d
import h5py as h5
from pymatgen.io.ase import AseAtomsAdaptor
#from unfold import make_kpath, removeDuplicateKpoints, find_K_from_k

dark_blue = (0, 0, .7, 1)
dark_blue_transparent = (0, 0, .7, .2)
dark_blue_semitransparent = (0, 0, .7, .5)
dark_red = (.8, 0, 0, 1)
dark_red_transparent = (.8, 0, 0, .2)
dark_red_semitransparent = (.8, 0, 0, .5)
dark_green = (0, .4, 0, 1)
dark_green_transparent = (0, .4, 0, .2)
dark_green_semitransparent = (0, .4, 0, .5)
dark_purple = (.8, 0, .7, 1)
dark_purple_transparent = (.8, 0, .7, .2)
dark_purple_semitransparent = (.8, 0, .7, .5)

def colorfunction_hue(phase, contrast = 1, brightness = 1): return(.5 * brightness + .499 * np.cos(phase) * contrast, .5 * brightness + .499 * np.cos(phase + 2 * np.pi / 3) * contrast, .5 * brightness + .499 * np.cos(phase + 4 * np.pi / 3) * contrast, 1)

def get_color_and_radius(specie):
    """Return the color and van der Waals radius for a given atomic specie."""
    match str(specie):
        case "W":
            vdW_radius = 2.49
            atom_color = [.6, .2, .6]
        case "Cr":
            vdW_radius = 1.39
            atom_color = [.5, .5, .5]
        case "Si":
            vdW_radius = 1.2
            atom_color = [.5, .5, .5]
        case "C":
            vdW_radius = 1.7
            atom_color = [.1, .1, .1]
        case "O":
            vdW_radius = 1.52
            atom_color = [1, 0, 0]
        case "H":
            vdW_radius = 1.2
            atom_color = [.9, .9, .9]
        case "N":
            vdW_radius = 2 * 1.55
            atom_color = [0, 0, 1]
        case "P":
            vdW_radius = 1.8
            atom_color = [1, 0.5, 0]
        case "B":
            vdW_radius = 1.92
            atom_color = [0.8, 0.5, 0.5]
        case "Fe":
            vdW_radius = 1.32
            atom_color = [0.5, 0, 0]
        case "S":
            vdW_radius = 1.8
            atom_color = [.9, .9, 0]
        case _:
            vdW_radius = 1.5
            atom_color = [.5, .5, .5]
    
    return atom_color, vdW_radius

def build_GNR(N = 2):
    xlist = np.zeros(N * 2, dtype = float)
    ylist = np.zeros(N * 2, dtype = float)
    atomlist = np.zeros(N * 2, dtype = str)
    CC_length = 1.42
    CH_length = 1.09
    sqrt3_2 = np.sqrt(3) / 2
    row_to_row = .5 * CC_length * sqrt3_2 # Row-to-row spacing in the y direction

    for i in range(0, N * 2, 2):
        xlist[i] = CC_length * (.25 * np.mod(i, 4) + .5) # Atom to the right side of the x axis
        xlist[i + 1] = -CC_length * (.25 * np.mod(i, 4) + .5) # Atom to the left side of the x axis
        ylist[i] = ylist[i + 1] = i * row_to_row
        atomlist[i] = "C"
        atomlist[i + 1] = "C"

    # Pad hydrogen atoms to the bottom and top of the GNR
    xlist = np.append(xlist, [.5 * CC_length + .5 * CH_length, -.5 * CC_length - .5 * CH_length,
                            -CC_length * (.25 * np.mod((N + 1) * 2, 4) + .5) + CH_length * (.5 - np.mod(N, 2)), CC_length * (.25 * np.mod((N + 1) * 2, 4) + .5) - CH_length * (.5 - np.mod(N, 2))])
    ylist = np.append(ylist, [-CH_length * np.sqrt(3) / 2, -CH_length * np.sqrt(3) / 2,
                            np.max(ylist) + CH_length * sqrt3_2, np.max(ylist) + CH_length * sqrt3_2])
    atomlist = np.append(atomlist, ["H", "H", "H", "H"])
    ylist -= np.mean(ylist) # Shift the y coordinates so that the structure is centered around the origin

    latvec_x = 3 * CC_length # Simply the unit cell length
    latvec_y = (N + 1) * 4 * row_to_row # The GNR hard wall boundary conditions dictate that the transverse component of the wavefunctions have nodal planes on atomic row 0 and atomic row N + 1
    # Thus, using this width ensures that the unit cell exactly fits the wavelength of the transverse waves and integer multiples of it
    # This prevents having to represent the transverse nodal plane structure with lots of different waves, also known as Fourier leakage
    latvec_z = 10 # 1 nm distance in the height direction should be sufficient to limit interactions between periodic images

    xlist_shifted = xlist + .5 * latvec_x # Move the atoms from being centered around the origin to being centered around the center of the unit cell
    ylist_shifted = ylist + .5 * latvec_y

    structure = Structure(lattice = Lattice.from_parameters(latvec_x, latvec_y, latvec_z, 90, 90, 90), species = atomlist,
                        coords = np.array([xlist_shifted, ylist_shifted, np.zeros_like(xlist) + .5 * latvec_z]).T, coords_are_cartesian = True)

    return structure

def create_2D_model(structure, atom_size = 0.25, bond_size = 0.9, max_bondlength = 1.8):
    site_list = structure.sites # These are the atoms that will be plotted
    n_atoms = len(site_list)

    atoms = []
    bonds = []
    
    for atom_i in range(n_atoms): # Loop over all the atoms
        site_i = site_list[atom_i] # Extract information
        specie_i = site_i.specie
        xyz_i = site_i.coords
        atom_color_i, vdW_radius_i = get_color_and_radius(specie_i)
        radius_i = atom_size * vdW_radius_i
        atom = plt.Circle((xyz_i[0], xyz_i[1]), radius_i, color = atom_color_i)
        atoms.append(atom) #ax.add_patch(circle)

        for atom_j in range(atom_i + 1, n_atoms): # Loop over atom pairs for bond detection
            site_j = site_list[atom_j]
            specie_j = site_j.specie
            xyz_j = site_j.coords
            vec_ij = xyz_j - xyz_i
            dist_ij = np.linalg.norm(vec_ij) # Detect the bond distance

            if dist_ij < max_bondlength and dist_ij > 1e-1: # Create a bond when the atoms are closer than the max bond distance
                atom_color_j, vdW_radius_j = get_color_and_radius(specie_j)
                normvec_ij = vec_ij / dist_ij # Calculate the normalized vector in order to determine the orientation of the bond in 3D space
                angle_ij = np.arctan(normvec_ij[1] / normvec_ij[0])
                
                xyz_avg = .5 * xyz_i + .5 * xyz_j
                bond_color = 0.5 * np.array(atom_color_i) + 0.5 * np.array(atom_color_j) # Set the bond color to be the average of the colors associated with the two atoms
                if str(specie_i) == "H" or str(specie_j) == "H": # If one of the atoms is a hydrogen atom, use a white bond instead
                    bond_color = np.array([0.8, 0.8, 0.8])
                height = bond_size * atom_size * 1.2
                bond = plt.Rectangle((xyz_avg[0] - .5 * dist_ij, xyz_avg[1] - .5 * height), dist_ij, height, angle = 180 * angle_ij / np.pi, rotation_point = "center", color = bond_color)
                bonds.append(bond) #ax.add_patch(bond)

    return atoms, bonds

def create_3D_model(structure, atom_size = 0.25, bond_size = 0.9, max_bondlength = 1.8, resolution = 20):
    site_list = structure.sites
    n_atoms = len(site_list)
    
    spheres = []
    cylinders = []

    for atom_i in range(n_atoms): # Loop over all the atoms
        site_i = site_list[atom_i] # Extract information
        specie_i = site_i.specie
        xyz_i = site_i.coords
        atom_color_i, vdW_radius_i = get_color_and_radius(specie_i)
        radius_i = atom_size * vdW_radius_i

        # Create a sphere for the current atom
        sphere = o3d.geometry.TriangleMesh.create_sphere(radius = radius_i, resolution = resolution)
        sphere.paint_uniform_color(atom_color_i)
        sphere.translate((xyz_i[0], xyz_i[1], xyz_i[2]))
        sphere.compute_vertex_normals()
        spheres += [sphere]

        for atom_j in range(atom_i + 1, n_atoms): # Loop over atom pairs for bond detection
            site_j = site_list[atom_j]
            specie_j = site_j.specie
            xyz_j = site_j.coords
            vec_ij = xyz_j - xyz_i
            dist_ij = np.linalg.norm(vec_ij) # Detect the bond distance
            
            if dist_ij < max_bondlength and dist_ij > 1e-1: # Create a bond when the atoms are closer than the max bond distance
                atom_color_j, vdW_radius_j = get_color_and_radius(specie_j)
                normvec_ij = vec_ij / dist_ij # Calculate the normalized vector in order to determine the orientation of the bond in 3D space
                bond_color = 0.5 * np.array(atom_color_i) + 0.5 * np.array(atom_color_j) # Set the bond color to be the average of the colors associated with the two atoms
                if str(specie_i) == "H" or str(specie_j) == "H": # If one of the atoms is a hydrogen atom, use a white bond instead
                    bond_color = np.array([0.8, 0.8, 0.8])

                # Create a cylinder for the bond
                z_axis = np.array([0, 0, 1])
                rotation_axis = np.cross(z_axis, normvec_ij) # Calculate the rotation matrix from the angle between the bond unit vector and the z axis
                if np.linalg.norm(rotation_axis) < 1e-8:
                    Rot_matrix = np.eye(3)
                else:
                    rotation_axis /= np.linalg.norm(rotation_axis)
                    rotation_angle = np.arccos(np.clip(np.dot(z_axis, normvec_ij), -1.0, 1.0))
                    Rot_matrix = o3d.geometry.get_rotation_matrix_from_axis_angle(rotation_axis * rotation_angle)
            
                cylinder = o3d.geometry.TriangleMesh.create_cylinder(radius = bond_size * atom_size * 1.2, height = dist_ij, resolution = resolution, split = 4)
                cylinder.paint_uniform_color(bond_color)
                cylinder.rotate(Rot_matrix, center = (0, 0, 0))
                cylinder.translate((xyz_i + xyz_j) / 2)
                cylinder.compute_vertex_normals()
                cylinders += [cylinder]
    
    return spheres, cylinders

def structure_plot(structure, supercell = (1, 1, 1), atom_size = 0.25, bond_size = 0.9, max_bondlength = 1.6, resolution = 20, dimensions = 2):
    """ Plot the structure as svg, so the orbitals can be overlaid """
    superstructure = structure.copy() # Create a superstructure, using the user-defined supercell parameters
    if supercell != (1, 1, 1) and type(supercell) == tuple and len(supercell) == 3: superstructure.make_supercell(supercell)
    superlattice = superstructure.lattice
    latvec_x = superlattice.a
    latvec_y = superlattice.b
    supercell_center = 0.5 * np.sum(superlattice.matrix, axis = 1)

    structure_ext = superstructure.copy() # Extend the structure beyond the unit cell to account for bonds that protrude through the unit cell boundaries
    structure_ext.make_supercell((3, 1, 1))
    structure_ext.translate_sites(vector = [-latvec_x, 0, 0], indices = range(len(structure_ext.sites)), frac_coords = False, to_unit_cell = False)
    remove_list = [] # Remove the atoms beyond the unit cell boundaries that do not create bonds
    for i in range(len(structure_ext.sites)):
        if not -max_bondlength < structure_ext.sites[i].coords[0] < latvec_x + max_bondlength: remove_list.append(i)
    structure_ext.remove_sites(remove_list)
    site_list = structure_ext.sites # These are the atoms that will be plotted

    if dimensions == 2:
        fig, ax = plt.subplots() # Create the plot
        plt.tight_layout()
        ax.set_aspect(1)
        ax.set_xbound([0, latvec_x])
        ax.set_ybound([0, latvec_y])
        atoms, bonds = create_2D_model(structure_ext, atom_size = atom_size, bond_size = bond_size, max_bondlength = max_bondlength)
        bonds_collection = PatchCollection(bonds, match_original = True)
        ax.add_collection(bonds_collection)
        atoms_collection = PatchCollection(atoms, match_original = True)
        ax.add_collection(atoms_collection)
        return fig
    
    elif dimensions == 3:
        atoms, bonds = create_3D_model(structure_ext, atom_size = atom_size, bond_size = bond_size, max_bondlength = max_bondlength)
        o3d.visualization.draw_geometries(bonds + atoms, lookat = supercell_center, zoom = 0.3, up = [0, 1, 0], width = 600, height = 400)

    else: print("Unknown dimensionality")

def procar_loader(folder_name, verbose = True):
    procar_data = vasp_out.Procar(folder_name + "PROCAR")
    structure_file_name = folder_name + "CONTCAR"
    structure, lat_vec, center = structure_loader(structure_file_name, show = False)

    n_bands = procar_data.nbands
    n_kpoints = procar_data.nkpoints
    n_ions = procar_data.nions
    n_noH = sum([int(str(structure.sites[i].specie) != "H") for i in range(len(structure.sites))]) # The number of atoms that are not hydrogen atoms
    spin_polarized = bool(procar_data.nspins - 1)
    
    orbital_list = procar_data.orbitals
    pz_index = int(np.where(np.asarray(orbital_list) == "pz")[0][0])
    n_orbitals = len(orbital_list)
    if verbose:
        print("Number of k points: " + str(n_kpoints) + "; number of bands: " + str(n_bands) + "; number of atoms: " + str(n_ions) + "; number of hydrogen atoms: " + str(n_ions - n_noH) + "; number of orbitals: " + str(n_orbitals))
        print("Identity of orbitals: ", orbital_list)

    CB_min_list = np.zeros(n_kpoints, dtype = float)
    VB_max_list = np.zeros(n_kpoints, dtype = float)

    for k_point in range(n_kpoints):
        luco_index = int(np.where(procar_data.occupancies[Spin.up][k_point] < .5)[0][0]) # Find the index of the eigenstate where the occupancy goes to zero
        CB_min_list[k_point] = procar_data.eigenvalues[Spin.up][k_point][luco_index]
        VB_max_list[k_point] = procar_data.eigenvalues[Spin.up][k_point][luco_index - 1]

    luco_energy = np.min(CB_min_list)
    hoco_energy = np.max(VB_max_list)
    midgap_energy = .5 * hoco_energy + .5 * luco_energy
    band_gap = luco_energy - hoco_energy

    if verbose:
        print("The VB maximum is at " + str(hoco_energy) + " eV and the CB minimum is at " + str(luco_energy) + " eV, giving a band gap of " + str(band_gap) + " eV")
        print("Spin polarized calculation: " + str(spin_polarized))

    return [n_bands, n_kpoints, n_ions, n_noH, n_orbitals, spin_polarized, orbital_list, pz_index, luco_index, hoco_energy, luco_energy, midgap_energy, band_gap, procar_data]

def wavecar_loader(folder_name, verbose = True):
    if not os.path.exists(os.path.dirname(folder_name)):
        print("Folder not found!")
        return
    if not os.path.exists(os.path.dirname(folder_name) + "/WAVECAR"):
        print("WAVECAR file not found!")
        return
    if not os.path.exists(os.path.dirname(folder_name) + "/CONTCAR"):
        print("Associated CONTCAR file not found!")
    else: structure = Structure.from_file(os.path.dirname(folder_name) + "/CONTCAR")

    wavecar_data = vasp_out.Wavecar(os.path.dirname(folder_name) + "/WAVECAR") # Loading WAVECAR files might take a few seconds
    n_bands = wavecar_data.nb
    n_kpoints = wavecar_data.nk
    spin_polarized = bool(wavecar_data.spin - 1)

    if verbose: print("Number of k points: " + str(n_kpoints) + "; number of bands: " + str(n_bands))

    energies = wavecar_data.band_energy
    n2_kpoints = n_kpoints
    if spin_polarized: # Put the spin up data and spin down data behind each other in k-space for checking the band minima and maxima
        energies = np.vstack(energies)
        n2_kpoints = 2 * n_kpoints
    CB_min_list = np.zeros(n2_kpoints, dtype = float)
    VB_max_list = np.zeros(n2_kpoints, dtype = float)

    for k_point in range(n2_kpoints):
        luco_index = int(np.where(energies[k_point][:, 2] < .5)[0][0]) # Find the index of the eigenstate where the occupancy goes to zero
        CB_min_list[k_point] = energies[k_point][luco_index, 0]
        VB_max_list[k_point] = energies[k_point][luco_index - 1, 0]

    luco_energy = np.min(CB_min_list)
    hoco_energy = np.max(VB_max_list)
    midgap_energy = .5 * hoco_energy + .5 * luco_energy
    band_gap = luco_energy - hoco_energy

    if verbose:
        print("The VB maximum is at " + str(hoco_energy) + " eV and the CB minimum is at " + str(luco_energy) + " eV, giving a band gap of " + str(band_gap) + " eV")
        print("Spin polarized calculation: " + str(spin_polarized))
    
    return [n_bands, n_kpoints, spin_polarized, luco_index, hoco_energy, luco_energy, midgap_energy, band_gap, wavecar_data, structure]

def get_bands(wavecar_data, midgap_energy = 0):
    n_kpoints = wavecar_data.nk
    n_bands = wavecar_data.nb
    spin_polarized = bool(wavecar_data.spin - 1)

    sigma_bands0 = np.zeros((2, n_kpoints, n_bands, 5), dtype = float) # Initialize arrays for the sigma and pi bands
    pi_bands0 = np.zeros_like(sigma_bands0) # The arrays are initialized too large but will later be truncated to fit the number of bands

    for spin in range(2):
        if spin_polarized:
            all_coeffs = wavecar_data.coeffs[spin]
            all_eigs = wavecar_data.band_energy[spin]
        else:
            all_coeffs = wavecar_data.coeffs
            all_eigs =  wavecar_data.band_energy

        for k_point in range(n_kpoints):
            g_points = wavecar_data.Gpoints[k_point]
            coeffs_at_k = all_coeffs[k_point]
            eigs_at_k = all_eigs[k_point]
            
            n_sigma = 0 # The number of pi and sigma bands is counted for each k point
            n_pi = 0

            for band in range(n_bands):
                eigenenergy = eigs_at_k[band][0] - midgap_energy
                coeffs_band = coeffs_at_k[band] # Read the coefficients

                # Check if the wfn has more weight on nx = 0 (meaning it is symmetric wrt reflection in the yz plane) or on nx = 1 (antisymmetric wrt reflection in the yz plane)
                #g_points_list = [[0., 1., 1.], [1., 1., 1.], [2., 1., 1.], [3., 1., 1.]] # All wfns have weight on ny = 1 and nz = 1 due to the modulation incurred from the vacuum regions
                #indices = [(int(np.where(np.all(g_points == g_point, axis = 1))[0][0])) for g_point in g_points_list]
                #coeffs = np.abs(np.array([wave_band[index] for index in indices]))
                #nx = int(np.where(coeffs == np.max(coeffs))[0][0])
                #if coeffs[0] > coeffs[1]: nx = 0
                nx = 1

                # Check the wfn sigma/pi symmetry
                g_points_list = [[nx * 1., 1., 1.], [nx * 1., 1., -1.], [2., 2., 1.], [2., 2., -1.]] # Opposite k-points (reflected in z) are retrieved at nx = 0 and nx = 1
                indices = [(int(np.where(np.all(g_points == g_point, axis = 1))[0][0])) for g_point in g_points_list]
                coeffs = np.array([coeffs_band[index] for index in indices])
                if np.abs(np.diff(coeffs))[0] + np.abs(np.diff(coeffs))[2] < 10 ** (-5): symz = 0
                else: symz = 1
                nz = symz # If the wfn is a sigma orbital, focus only on wfns with nz = 0; otherwise, focus on wfns with nz = 1

                if nz < .5:
                    sigma_bands0[spin, k_point, n_sigma, 0] = eigenenergy
                    n_sigma += 1
                else: # For pi orbitals, determine more symmetries
                    pi_bands0[spin, k_point, n_pi, 0] = eigenenergy
                                        
                    g_points_list_ny = [[nx * 1., ny * 1., nz * 1.] for ny in range(8)]
                    g_points_list_minus_ny = [[nx * 1., -ny * 1., nz * 1.] for ny in range(8)]
                    indices_ny = [(int(np.where(np.all(g_points == g_point, axis = 1))[0][0])) for g_point in g_points_list_ny]
                    indices_minus_ny = [(int(np.where(np.all(g_points == g_point, axis = 1))[0][0])) for g_point in g_points_list_minus_ny]
                    coeffs_ny = np.array([coeffs_band[index] for index in indices_ny])
                    coeffs_minus_ny = np.array([coeffs_band[index] for index in indices_minus_ny])
                    sinlist = np.abs(np.array([coeffs_ny[ny] - coeffs_minus_ny[ny] for ny in range(8)]))
                    coslist = np.abs(np.array([coeffs_ny[ny] + coeffs_minus_ny[ny] for ny in range(8)]))
                    symy = np.sum(sinlist) / (np.sum(sinlist) + np.sum(coslist))
                    
                    pi_bands0[spin, k_point, n_pi, 1] = symy
                    n_pi += 1

    n_sigma_bands = np.min([np.array([np.where(np.diff(sigma_bands0[0, k_point, :, 0]) < 0.)[0][0] for k_point in range(n_kpoints)]),
                            np.array([np.where(np.diff(sigma_bands0[1, k_point, :, 0]) < 0.)[0][0] for k_point in range(n_kpoints)])]) # Determine where to crop the band arrays
    n_pi_bands = np.min([np.array([np.where(np.diff(pi_bands0[0, k_point, :, 0]) < 0.)[0][0] for k_point in range(n_kpoints)]),
                        np.array([np.where(np.diff(pi_bands0[1, k_point, :, 0]) < 0.)[0][0] for k_point in range(n_kpoints)])])
    sigma_bands = sigma_bands0[:, :, :n_sigma_bands]
    pi_bands = pi_bands0[:, :, :n_pi_bands]
    
    return sigma_bands, pi_bands

def bandstructure_plot(sigma_bands, pi_bands, n_kpoints, whichspin = 0, energy_range = [-7, 7], gamma = .01, de = .05):
    n_sigma_bands = len(sigma_bands[0, 0])
    n_pi_bands = len(pi_bands[0, 0])

    fig, ax = plt.subplots()
    fig.set_size_inches([4.8, 3.6], forward = True)
    plt.tight_layout()
    ax.set_xticks([-.75, -.5, 0, .5])
    ax.set_xticklabels(["pDOS", "-X", "Γ", "X"])
    ax.set_aspect(.1)
    ax.axvline(x = -.5, color = 'k')
    ax.axvline(x = 0, color = 'gray')
    ax.axvline(x = .5, color = 'gray')

    # For simple uninterrupted bands, use pyplot
    k_points = np.linspace(0, .5, n_kpoints)
    [ax.plot(-k_points, sigma_bands[whichspin, :, i, 0], c = dark_green_transparent) for i in range(n_sigma_bands)] # Plot the sigma bands to the left of Gamma (Spin 0, all k points, band by band, eigenenergy slot)
    [ax.plot(k_points, sigma_bands[whichspin, :, i, 0], c = dark_green_transparent) for i in range(n_sigma_bands)] # Plot them transparently to the right of Gamma
    #[ax.plot(-k_points, pi_bands[whichspin, :, i, 0], c = dark_red) for i in range(n_pi_bands)] # Plot the pi bands
    #[ax.plot(k_points, pi_bands[whichspin, :, i, 0], c = dark_red) for i in range(n_pi_bands)]
    #[ax.plot(1 - k_points, pi_bands[whichspin, :, i, 0], c = dark_red) for i in range(n_pi_bands)] # Plot the pi bands in the second BZ

    # For projections, use line collection
    for band in range(n_pi_bands):
        segments = [[[k_points[i], pi_bands[whichspin, i, band, 0]], [k_points[i + 1], pi_bands[whichspin, i + 1, band, 0]]] for i in range(n_kpoints - 1)]
        colors = []
        for i in range(n_kpoints - 1):
            avg_coeffs = np.average(pi_bands[whichspin, i:i + 2, band], axis = 0)
            colors.append((.8 * avg_coeffs[1], 0, .7 * (1 - avg_coeffs[1]), 1))
        lc = LineCollection(segments = segments, colors = colors)
        ax.add_collection(lc)
        segments = [[[-k_points[i], pi_bands[whichspin, i, band, 0]], [-k_points[i + 1], pi_bands[whichspin, i + 1, band, 0]]] for i in range(n_kpoints - 1)]
        lc = LineCollection(segments = segments, colors = colors)
        ax.add_collection(lc)
        
    # Calculate the projected DOS
    e_list = np.arange(energy_range[0], energy_range[1], de)
    dos_sigma = np.zeros_like(e_list)
    dos_pi = np.zeros_like(e_list)
    gamma2 = gamma ** 2

    for i in range(len(e_list)): # Loop over energy points
        en_diffs2 = (np.array([[(sigma_bands[whichspin, k_point, band, 0] - e_list[i]) ** 2 for band in range(n_sigma_bands)] for k_point in range(n_kpoints)])).flatten() # Calculate the distances between the energy point and the eigenenergies
        en_diffs2_sel = [en_diff2 for en_diff2 in en_diffs2 if en_diff2 < 90] # Ditch eigenenergies that are too far away and have a negligible contribution to the DOS
        dos_sigma[i] = sum([(gamma / np.pi) * (1  / (en_diff2 ** 2 + gamma2)) for en_diff2 in en_diffs2_sel]) # Sum over Lorentzian functions to determine the DOS as the energy point
        en_diffs2 = (np.array([[(pi_bands[whichspin, k_point, band, 0] - e_list[i]) ** 2 for band in range(n_pi_bands)] for k_point in range(n_kpoints)])).flatten() # Do this again, but now for the pi states
        en_diffs2_sel = [en_diff2 for en_diff2 in en_diffs2 if en_diff2 < 90]
        dos_pi[i] = sum([(gamma / np.pi) * (1  / (en_diff2 ** 2 + gamma2)) for en_diff2 in en_diffs2_sel])

    norm_dos = 2 * np.max(dos_sigma + dos_pi)
    dos_pi /= norm_dos
    dos_sigma /= norm_dos

    ax.fill_betweenx(e_list, - 0.5 - dos_pi - dos_sigma, - 0.5 - dos_pi, color = dark_green_semitransparent) # Color the sigma DOS blue
    ax.fill_betweenx(e_list, - 0.5 - dos_pi, - 0.5, color = dark_purple) # Color the pi DOS red
    ax.set_xbound([-1, .6])
    ax.set_ybound(energy_range)
    #plt.savefig(folder_name + "BS_full.svg")
    return fig

def get_Fourier_cube(wavecar_data, k_point = 0, band = 0, whichspin = 0, verbose = True):
    if k_point > wavecar_data.nk - 1:
        print(f"Error! Selected k-point ({k_point}) out of range (total number of k-points: {wavecar_data.nk})")
        return [[[]]]
    if band > wavecar_data.nb - 1:
        print(f"Error! Selected band index ({band}) out of range (total number of bands: {wavecar_data.nb})")
        return [[[]]]

    spin_polarized = bool(wavecar_data.spin - 1)
    upordown = "up"
    if whichspin == 1: upordown = "down"

    # Read wavecar data at the k-point
    g_points = wavecar_data.Gpoints[k_point]
    if spin_polarized: eigs = wavecar_data.band_energy[whichspin][k_point]
    else: eigs = wavecar_data.band_energy[k_point]
    
    nx_max = int(np.max(np.array([g_points[i][0] for i in range(len(g_points))])))
    ny_max = int(np.max(np.array([g_points[i][1] for i in range(len(g_points))])))
    nz_max = int(np.max(np.array([g_points[i][2] for i in range(len(g_points))])))

    # Read data for the specific band
    if spin_polarized: wave_band = wavecar_data.coeffs[whichspin][k_point][band]
    else: wave_band = wavecar_data.coeffs[k_point][band]

    # Check if the wfn has more weight on nx = 0 (meaning it is symmetric wrt reflection in the yz plane) or on nx = 1 (antisymmetric wrt reflection in the yz plane)
    g_points_list = [[0., 1., 1.], [1., 1., 1.]] # All wfns have weight on ny = 1 and nz = 1 due to the modulation incurred from the vacuum regions
    indices = [(int(np.where(np.all(g_points == g_point, axis = 1))[0][0])) for g_point in g_points_list]
    coeffs = np.abs(np.array([wave_band[index] for index in indices]))
    if coeffs[0] > coeffs[1]: nx = 0
    else: nx = 1

    # Check the wfn sigma/pi symmetry
    g_points_list = [[nx * 1., 1., 1.], [nx * 1., 1., -1.]] # Opposite k-points (reflected in z) are retrieved at nx = 0 and nx = 1
    indices = [(int(np.where(np.all(g_points == g_point, axis = 1))[0][0])) for g_point in g_points_list]
    coeffs = np.array([wave_band[index] for index in indices])
    if np.abs(np.diff(coeffs)) < 10 ** (-6): symz = 0
    else: symz = 1
    nz = symz # If the wfn is a sigma orbital, focus only on wfns with nz = 0; otherwise, focus on wfns with nz = 1

    if verbose:
        if nz == 0: print(f"Orbital number {band} with spin {whichspin} (spin {upordown}) at k-point {k_point} and energy {eigs[band][0]} eV. This orbital has sigma symmetry")
        else: print(f"Orbital number {band} with spin {whichspin} (spin {upordown}) at k-point {k_point} and energy {eigs[band][0]} eV. This orbital has pi symmetry")
    
    # Read in the coefficients
    Fourier_cube = np.zeros((2 * nx_max + 1, 2 * ny_max + 1, 2 * nz_max + 1), dtype = complex)
    for i in range(len(g_points)):
        nx, ny, nz = [int(g_points[i, dim]) for dim in range(3)]
        coeff = wave_band[i]
        Fourier_cube[nx, ny, nz] = coeff

    return Fourier_cube

def get_wfn(wavecar_data, k_point = 0, band = 0, normalize = True, phase_autozero = True, n_bins = 100, verbose = True):
    Fourier_cube = get_Fourier_cube(wavecar_data, k_point = k_point, band = band, verbose = verbose)
    if len(Fourier_cube) < 2: return [[[]]]
    wfn_cube = np.fft.ifftn(Fourier_cube)
    wfn2_cube = np.abs(wfn_cube) ** 2
    max_value = np.max(wfn2_cube)
    norm_fac = np.sqrt(max_value)
    
    if normalize and max_value > .0000000000001:
        wfn_cube /= (norm_fac + .0000000000001)
        wfn2_cube = np.abs(wfn_cube) ** 2
    
    wfnarg_cube = np.angle(wfn_cube)

    if phase_autozero:
        arg_histogram = np.histogram(wfnarg_cube.flatten(), bins = n_bins, range = [-np.pi, np.pi])
        max_index = np.where(arg_histogram[0] == np.max(arg_histogram[0]))[0][0]
        max_represented_phase = max_index * (2 * np.pi) / n_bins - np.pi
        wfnarg_cube = wfnarg_cube - max_represented_phase

    return wfn_cube, wfn2_cube, wfnarg_cube

def wfn_slice(wavecar_data, wfn_cube, slice_height = 5):
    lat_vec_z = wavecar_data.a[2, 2]
    grid_z = np.shape(wfn_cube)[2]
    Angstrom_per_voxel_z = lat_vec_z / grid_z
    slice_index = int(round(slice_height / Angstrom_per_voxel_z))
    wfn_rect = np.flip(wfn_cube[:, :, slice_index])

    return slice_index, wfn_rect

def orbital_plots(wavecar_data, k_point = 0, band = 0, height = 5., cells = [3, 1], zero_phase = 2.6, whichspin = 0, brightness = 1, normalize_slice = True, type = "density", threshold = .1, zoom_factor = 3, verbose = True):
    folder_name = os.path.dirname(wavecar_data.filename) + "/"
    file_name_struc = folder_name + "/CONTCAR"
    structure = Structure.from_file(file_name_struc)
    height_relative_pm = round(100 * (height - np.average(structure.frac_coords[:, 2]) * structure.lattice.c))

    wfn_folder_name = folder_name + "wfn_data"
    try:
        os.makedirs(wfn_folder_name, exist_ok=True)
        print(f"Directory {wfn_folder_name} created successfully or already exists.")
    except OSError as e: print(f"Error creating directory {directory_path}: {e}")
    plot_basename = wfn_folder_name + "/orb_" + str(band) + "_k_" + str(k_point) + "_spin_" + str(whichspin) + "_height_" + str(height_relative_pm) + "_pm_"

    Fourier_cube = get_Fourier_cube(wavecar_data, k_point = k_point, band = band, whichspin = whichspin, verbose = True)
    wfn_cube = np.fft.ifftn(Fourier_cube)
    wfn2_cube = np.abs(wfn_cube) ** 2
    max_value = np.max(wfn2_cube)
    norm_fac = np.sqrt(max_value)
    if max_value > .0000000000001:
        wfn_cube /= (norm_fac + .0000000000001)
        wfn2_cube = np.abs(wfn_cube) ** 2
    wfnarg_cube = np.angle(wfn_cube)

    arg_histogram = np.histogram(wfnarg_cube.flatten(), bins = 100, range = [-np.pi, np.pi])
    max_index = np.where(arg_histogram[0] == np.max(arg_histogram[0]))[0][0]
    max_represented_phase = max_index * (2 * np.pi) / 100 - np.pi
    wfnarg_cube = wfnarg_cube - max_represented_phase

    slice_index, wfn_rect = wfn_slice(wavecar_data, wfn_cube, slice_height = height) # Get 2D slices of the 3D data
    slice_index, wfn2_rect = wfn_slice(wavecar_data, wfn2_cube, slice_height = height)
    slice_index, wfnarg_rect = wfn_slice(wavecar_data, wfnarg_cube, slice_height = height)

    # Calculate the standard orbital
    if normalize_slice: wfn2_rect /= np.max(wfn2_rect)
    arg_img = np.array([[colorfunction_hue(wfnarg_rect[i, j] + zero_phase) for i in range(len(wfnarg_rect))] for j in range(len(wfnarg_rect[0]))]) # Create the color image from the wavefunction arguments
    orbital_img = np.array([[(arg_img[j, i, 0], arg_img[j, i, 1], arg_img[j, i, 2], wfn2_rect[i, j]) for i in range(len(wfn2_rect))] for j in range(len(wfn2_rect[0]))])

    zoomed_orbital = np.clip(zoom(orbital_img, (zoom_factor, zoom_factor, 1)), a_min = 0, a_max = 1)
    zoomed_orbital = np.vstack([np.hstack([zoomed_orbital for _ in range(cells[0])]) for _ in range(cells[1])])
    plt.imshow(zoomed_orbital)
    plt.tight_layout()
    plt.show()
    plt.imsave(plot_basename + "wfn.png", zoomed_orbital)

    # Calculate the orbital as gauged with a pi-wave tip (CO tip)
    dwfn_rect_dx = convolve2d(wfn_rect, np.array([[1, -1], [1, -1]], dtype = float), boundary = "wrap")
    diwfn_rect_dy = convolve2d(wfn_rect, np.array([[1, 1], [-1, -1]], dtype = float), boundary = "wrap")
    dwfn_rect_dxdy = dwfn_rect_dx + 1j * diwfn_rect_dy
    piwfn2_rect = np.abs(dwfn_rect_dxdy) ** 2
    piwfn2_rect /= np.max(piwfn2_rect)
    piwfnarg_rect = np.angle(dwfn_rect_dxdy)
    piarg_img = np.array([[colorfunction_hue(piwfnarg_rect[i, j] + zero_phase) for i in range(len(piwfnarg_rect))] for j in range(len(piwfnarg_rect[0]))]) # Create the color image from the wavefunction arguments
    piorbital_img = np.array([[(piarg_img[j, i, 0], piarg_img[j, i, 1], piarg_img[j, i, 2], piwfn2_rect[i, j]) for i in range(len(piwfn2_rect))] for j in range(len(piwfn2_rect[0]))])

    zoomed_piorbital = np.clip(zoom(piorbital_img, (zoom_factor, zoom_factor, 1)), a_min = 0, a_max = 1)
    zoomed_piorbital = np.vstack([np.hstack([zoomed_piorbital for _ in range(cells[0])]) for _ in range(cells[1])])
    plt.imshow(zoomed_piorbital)
    plt.tight_layout()
    plt.show()
    plt.imsave(plot_basename + "wfn_COtip.png", zoomed_piorbital)

    # Calculate simulated maps
    orbital_sigma_tip = np.vstack([np.hstack([zoom(wfn2_rect.T, (zoom_factor, zoom_factor)) for _ in range(cells[0])]) for _ in range(cells[1])])
    orbital_pi_tip = np.vstack([np.hstack([zoom(piwfn2_rect.T, (zoom_factor, zoom_factor)) for _ in range(cells[0])]) for _ in range(cells[1])])
    plt.imshow(orbital_sigma_tip, cmap = "gray")
    plt.tight_layout()
    plt.show()
    plt.imsave(plot_basename + "orb.png", orbital_sigma_tip, cmap = "gray")
    plt.imshow(orbital_pi_tip, cmap = "gray")
    plt.tight_layout()
    plt.show()
    plt.imsave(plot_basename + "orb_COtip.png", orbital_pi_tip, cmap = "gray")

def orbital_info(procar_data, k_point = 0, band = 0):
    orbital_list = procar_data.orbitals
    proj_matrix = procar_data.data[Spin.up][k_point][band]
    projections = sum([proj_matrix[atom] for atom in range(len(proj_matrix))])
    occupancy = procar_data.occupancies[Spin.up][k_point][band]
    pz_index = int(np.where(np.asarray(orbital_list) == "pz")[0][0])
    pz_weight = projections[pz_index]
    eigenenergy = procar_data.eigenvalues[Spin.up][k_point][band]

    if occupancy > .5:
        if pz_weight > .5: print(f"Orbital {band} is an occupied pi orbital with an energy of {eigenenergy} eV.")
        else: print(f"Orbital {band} is an occupied sigma orbital with an energy of {eigenenergy} eV.")
    else:
        if pz_weight > .5: print(f"Orbital {band} is an unoccupied pi orbital with an energy of {eigenenergy} eV.")
        else: print(f"Orbital {band} is an unoccupied sigma orbital with an energy of {eigenenergy} eV.")

np.set_printoptions(suppress = True)

Simple visualization tool

In [188]:
file_name = "C:/DFT/5-AGNR/5-AGNR_12_B_row2/POSCAR"

structure = Structure.from_file(file_name)

structure_plot(structure, dimensions = 3)

Simple calculation of the number of atoms ("ions") and electrons in a structure

In [ ]:
file_name = "C:/DFT/4-AGNR/4-AGNR_8_N_row2/CONTCAR"

structure = Structure.from_file(file_name)

atoms = structure.sites
n_elec = 0

for atom in atoms:
    if atom.species_string == "H":
        n_elec += 1
    elif atom.species_string == "C":
        n_elec += 4
    elif atom.species_string == "N":
        n_elec += 5
    elif atom.species_string == "B":
        n_elec += 3

print("Number of atoms:", len(atoms))
print("Number of electrons:", n_elec)

Construct GNRs

In [ ]:
N_ribbon = 2

file_name = "C:/DFT/" + str(N_ribbon) + "-AGNR_primitive_POSCAR" # Create the POSCAR file
structure = build_GNR(N = N_ribbon)
#structure.to(filename = file_name, fmt = "poscar")
structure_plot(structure)

Band structure from PROCAR file

In [ ]:
N_ribbon = 2

#folder_name = "C:/DFT/" + str(N_ribbon) + "-AGNR/" + str(N_ribbon) + "-AGNR_primitive/"
folder_name = "C:/DFT/" + str(N_ribbon) + "-AGNR/"
folder_name = "C:/DFT/4-AGNR/4-AGNR_8_N_row2/"

file_name = folder_name + "CONTCAR" # Get the CONTCAR for visualization and determination of the atoms (species and coordinates)

structure, lat_vec, center = structure_loader(file_name)
n_bands, n_kpoints, n_ions, n_noH, n_orbitals, spin_polarized, orbital_list, pz_index, luco_index, hoco_energy, luco_energy, midgap_energy, band_gap, procar_data = procar_loader(folder_name)

Number of k points: 5; number of bands: 368; number of atoms: 96; number of hydrogen atoms: 32; number of orbitals: 9
Identity of orbitals:  ['s', 'py', 'pz', 'px', 'dxy', 'dyz', 'dz2', 'dxz', 'x2-y2']
The VB maximum is at -2.04061611 eV and the CB minimum is at -1.5053 eV, giving a band gap of 0.5353161100000001 eV
Spin polarized calculation: True


In [20]:
midgap_to_zero = True

sigma_bands_up0 = np.zeros((n_kpoints, n_bands, 5), dtype = float) # Initialize arrays for the sigma and pi bands
pi_bands_up0 = sigma_bands_down0 = pi_bands_down0 = np.zeros_like(sigma_bands_up0) # The arrays are initialized too large but will later be truncated to fit the number of bands

for k_point in range(n_kpoints): # Loop over all k-points
    if spin_polarized:
        up_occupancies = procar_data.occupancies[Spin.up][k_point] # Read the orbital occupations
        down_occupancies = procar_data.occupancies[Spin.down][k_point] # Read spin down occupancies only if the calculation is spin-polarized
    else:
        up_occupancies = procar_data.occupancies[Spin.up][k_point] / 2
        down_occupancies = up_occupancies

    if midgap_to_zero: up_data = procar_data.eigenvalues[Spin.up][k_point] - midgap_energy # Read the eigenvalues
    else: up_data = procar_data.eigenvalues[Spin.up][k_point]
    up_proj_matrix = procar_data.data[Spin.up][k_point] # Read the projections
    down_data = up_data
    down_proj_matrix = up_proj_matrix
    if spin_polarized:
        if midgap_to_zero: down_data = procar_data.eigenvalues[Spin.down][k_point] - midgap_energy
        else: down_data = procar_data.eigenvalues[Spin.down][k_point]
        down_proj_matrix = procar_data.data[Spin.down][k_point]

    n_sigma_up = 0
    n_pi_up = 0
    n_sigma_down = 0
    n_pi_down = 0

    for band in range(n_bands): # Loop over the bands
        eigenenergy = up_data[band]
        occupancy = up_occupancies[band]
        norm_fac = np.sum(up_proj_matrix[band]) + .000000001 # Normalize the total of the projection matrix. The offset is to prevent accidental divisions by zero
        noH_ions = sum([up_proj_matrix[band][atom] for atom in range(n_noH)]) # Sum projections over atoms that are not hydrogen
        H_ions = sum(sum([up_proj_matrix[band][atom] for atom in range(n_noH, n_ions)])) / norm_fac # Sum projections over atoms that are hydrogen
        noH_ions_pz = noH_ions[pz_index] / norm_fac # The projection found at the pz_index'th slot is the projection on the pz orbitals
        noH_ions_other = sum([noH_ions[i] for i in list(set(range(n_orbitals)).symmetric_difference(set([2])))]) / norm_fac # The rest of the orbitals are not pz

        if noH_ions_pz < noH_ions_other + H_ions:
            sigma_bands_up0[k_point, n_sigma_up, 0] = eigenenergy
            sigma_bands_up0[k_point, n_sigma_up, 1] = occupancy
            n_sigma_up += 1
        else:
            pi_bands_up0[k_point, n_pi_up, 0] = eigenenergy
            pi_bands_up0[k_point, n_pi_up, 1] = occupancy
            n_pi_up += 1

        energy = down_data[band]
        occupancy = down_occupancies[band]
        norm_fac = np.sum(down_proj_matrix[band]) + .000000001
        noH_ions = sum([down_proj_matrix[band][atom] for atom in range(n_noH)])
        H_ions = sum(sum([down_proj_matrix[band][atom] for atom in range(n_noH, n_ions)])) / norm_fac
        noH_ions_pz = noH_ions[pz_index] / norm_fac
        noH_ions_other = sum([noH_ions[i] for i in list(set(range(n_orbitals)).symmetric_difference(set([2])))]) / norm_fac

        if noH_ions_pz < noH_ions_other + H_ions:
            sigma_bands_down0[k_point, n_sigma_down, 0] = eigenenergy
            sigma_bands_down0[k_point, n_sigma_down, 1] = occupancy
            n_sigma_down += 1
        else:
            pi_bands_down0[k_point, n_pi_down, 0] = eigenenergy
            pi_bands_down0[k_point, n_pi_down, 1] = occupancy
            n_pi_down += 1

n_sigma_bands_up = np.min(np.array([np.where(np.diff(sigma_bands_up0[k_point, :, 0]) < 0.)[0][0] for k_point in range(n_kpoints)])) # Determine where to crop the band arrays
n_pi_bands_up = np.min(np.array([np.where(np.diff(pi_bands_up0[k_point, :, 0]) < 0.)[0][0] for k_point in range(n_kpoints)]))
sigma_bands_up = np.array([[sigma_bands_up0[:, i, 0], sigma_bands_up0[:, i, 1], np.ones_like(sigma_bands_up0[:, i, 0])] for i in range(n_sigma_bands_up)])
pi_bands_up = np.array([[pi_bands_up0[:, i, 0], pi_bands_up0[:, i, 1], np.ones_like(sigma_bands_up0[:, i, 0])] for i in range(n_pi_bands_up)])

n_sigma_bands_down = np.min(np.array([np.where(np.diff(sigma_bands_down0[k_point, :, 0]) < 0.)[0][0] for k_point in range(n_kpoints)])) # Determine where to crop the band arrays
n_pi_bands_down = np.min(np.array([np.where(np.diff(pi_bands_down0[k_point, :, 0]) < 0.)[0][0] for k_point in range(n_kpoints)]))
sigma_bands_down = np.array([[sigma_bands_down0[:, i, 0], sigma_bands_down0[:, i, 1], np.ones_like(sigma_bands_down0[:, i, 0])] for i in range(n_sigma_bands_down)])
pi_bands_down = np.array([[pi_bands_down0[:, i, 0], pi_bands_down0[:, i, 1], np.ones_like(sigma_bands_down0[:, i, 0])] for i in range(n_pi_bands_down)])

Band structure from WAVECAR file

In [ ]:
N_GNR = 5
GNR_orientation = "A" # "A" for armchair or "Z" for zigzag
defect_row = 3
defect_type = "N" # Swap to something else for primitive cell calculations
supercell_length = 12

if defect_type != "N" and defect_type != "B": folder_name = "C:/DFT/" + str(N_GNR) + "-" + GNR_orientation + "GNR/" + str(N_GNR) + "-" + GNR_orientation + "GNR_primitive/" # Primitive GNRS
else: folder_name = "C:/DFT/" + str(N_GNR) + "-" + GNR_orientation + "GNR/" + str(N_GNR) + "-" + GNR_orientation + "GNR_" + str(supercell_length) + "_" + str(defect_type) + "_row" + str(defect_row) + "/"

print(folder_name)
n_bands, n_kpoints, spin_polarized, luco_index, hoco_energy, luco_energy, midgap_energy, band_gap, wavecar_data, structure = wavecar_loader(folder_name)
plot = structure_plot(structure, dimensions = 2)
plt.show()

Plot the band structure. Band structure data calculated from the PROCAR or the WAVECAR file should give the same result

In [595]:
sigma_bands, pi_bands = get_bands(wavecar_data, midgap_energy)

In [ ]:
energy_range = [-7, 7]
de = .05 # Resolution for DOS calculation
gamma = .01 # Lorentzian width for broadening
whichspin = 0

bs = bandstructure_plot(sigma_bands, pi_bands, n_kpoints, whichspin = whichspin, energy_range = energy_range, gamma = gamma, de = de)
bs.savefig(folder_name + "BS_full.svg")
plt.show()

In [ ]:
n_kpoints = wavecar_data.nk
n_bands = wavecar_data.nb
spin_polarized = bool(wavecar_data.spin - 1)

sigma_bands0 = np.zeros((2, n_kpoints, 10, 5), dtype = float) # Initialize arrays for the sigma and pi bands
pi_bands0 = np.zeros_like(sigma_bands0) # The arrays are initialized too large but will later be truncated to fit the number of bands

for spin in range(2):
    if spin_polarized:
        all_coeffs = wavecar_data.coeffs[spin]
        all_eigs = wavecar_data.band_energy[spin]
    else:
        all_coeffs = wavecar_data.coeffs
        all_eigs =  wavecar_data.band_energy

    for k_point in range(n_kpoints):
        g_points = wavecar_data.Gpoints[k_point]
        coeffs_at_k = all_coeffs[k_point]
        eigs_at_k = all_eigs[k_point]
        
        n_sigma = 0 # The number of pi and sigma bands is counted for each k point
        n_pi = 0

        for band in range(200, 203): # n_bands
            if midgap_to_zero: eigenenergy = eigs_at_k[band][0] - midgap_energy
            else: eigenenergy = eigs_at_k[band][0]
            coeffs_band = coeffs_at_k[band] # Read the coefficients

            # Check if the wfn has more weight on nx = 0 (meaning it is symmetric wrt reflection in the yz plane) or on nx = 1 (antisymmetric wrt reflection in the yz plane)
            #g_points_list = [[0., 1., 1.], [1., 1., 1.], [2., 1., 1.], [3., 1., 1.]] # All wfns have weight on ny = 1 and nz = 1 due to the modulation incurred from the vacuum regions
            #indices = [(int(np.where(np.all(g_points == g_point, axis = 1))[0][0])) for g_point in g_points_list]
            #coeffs = np.abs(np.array([wave_band[index] for index in indices]))
            #nx = int(np.where(coeffs == np.max(coeffs))[0][0])
            #if coeffs[0] > coeffs[1]: nx = 0
            nx = 1

            # Check the wfn sigma/pi symmetry
            g_points_list = [[nx * 1., 1., 1.], [nx * 1., 1., -1.], [2., 2., 1.], [2., 2., -1.]] # Opposite k-points (reflected in z) are retrieved at nx = 0 and nx = 1
            indices = [(int(np.where(np.all(g_points == g_point, axis = 1))[0][0])) for g_point in g_points_list]
            coeffs = np.array([coeffs_band[index] for index in indices])
            if np.abs(np.diff(coeffs))[0] + np.abs(np.diff(coeffs))[2] < 10 ** (-5): symz = 0
            else: symz = 1
            nz = symz # If the wfn is a sigma orbital, focus only on wfns with nz = 0; otherwise, focus on wfns with nz = 1

            g_points_list_ny = [[nx * 1., ny * 1., nz * 1.] for ny in range(8)]
            g_points_list_minus_ny = [[nx * 1., -ny * 1., nz * 1.] for ny in range(8)]
            indices_ny = [(int(np.where(np.all(g_points == g_point, axis = 1))[0][0])) for g_point in g_points_list_ny]
            indices_minus_ny = [(int(np.where(np.all(g_points == g_point, axis = 1))[0][0])) for g_point in g_points_list_minus_ny]
            coeffs_ny = np.array([coeffs_band[index] for index in indices_ny])
            coeffs_minus_ny = np.array([coeffs_band[index] for index in indices_minus_ny])
            sinlist = np.abs(np.array([coeffs_ny[ny] - coeffs_minus_ny[ny] for ny in range(8)]))
            coslist = np.abs(np.array([coeffs_ny[ny] + coeffs_minus_ny[ny] for ny in range(8)]))
            symy = np.sum(sinlist) / (np.sum(sinlist) + np.sum(coslist))

            # Unfolding in x
            g_points_list = [[nx * 1., 0., 0.] for nx in range(-8, 8)]
            indices = [(int(np.where(np.all(g_points == g_point, axis = 1))[0][0])) for g_point in g_points_list]
            coeffs = np.array([coeffs_band[index] for index in indices])

            if nz < .5:
                sigma_bands0[spin, k_point, n_sigma, 0] = eigenenergy
                sigma_bands0[spin, k_point, n_sigma, 1] = symy
                n_sigma += 1
            else:
                pi_bands0[spin, k_point, n_pi, 0] = eigenenergy
                pi_bands0[spin, k_point, n_pi, 1] = symy
                n_pi += 1

n_sigma_bands = 3#np.min([np.array([np.where(np.diff(sigma_bands0[0, k_point, :, 0]) < 0.)[0][0] for k_point in range(n_kpoints)]),
                        #np.array([np.where(np.diff(sigma_bands0[1, k_point, :, 0]) < 0.)[0][0] for k_point in range(n_kpoints)])]) # Determine where to crop the band arrays
n_pi_bands = 3#np.min([np.array([np.where(np.diff(pi_bands0[0, k_point, :, 0]) < 0.)[0][0] for k_point in range(n_kpoints)]),
                    #np.array([np.where(np.diff(pi_bands0[1, k_point, :, 0]) < 0.)[0][0] for k_point in range(n_kpoints)])])
sigma_bands = sigma_bands0[:, :, :n_sigma_bands]
pi_bands = pi_bands0[:, :, :n_pi_bands]

Orbital plotting

In [ ]:
N_GNR = 5
GNR_orientation = "A" # "A" for armchair or "Z" for zigzag
defect_row = 2
defect_type = "N" # Swap to something else for primitive cell calculations
supercell_length = 12

if defect_type != "N" and defect_type != "B": folder_name = "C:/DFT/" + str(N_GNR) + "-" + GNR_orientation + "GNR/" + str(N_GNR) + "-" + GNR_orientation + "GNR_primitive/" # Primitive GNRS
else: folder_name = "C:/DFT/" + str(N_GNR) + "-" + GNR_orientation + "GNR/" + str(N_GNR) + "-" + GNR_orientation + "GNR_" + str(supercell_length) + "_" + str(defect_type) + "_row" + str(defect_row) + "/"

#n_bands, n_kpoints, n_ions, n_noH, n_orbitals, spin_polarized, orbital_list, pz_index, luco_index, hoco_energy, luco_energy, midgap_energy, band_gap, procar_data = procar_loader(folder_name)
n_bands, n_kpoints, spin_polarized, luco_index, hoco_energy, luco_energy, midgap_energy, band_gap, wavecar_data, structure = wavecar_loader(folder_name)
print(luco_index)
print(wavecar_data.efermi)

Number of k points: 1; number of bands: 448
The VB maximum is at -2.4866459963083978 eV and the CB minimum is at -2.4842328529624957 eV, giving a band gap of 0.0024131433459020357 eV
Spin polarized calculation: True
264
-2.485439424634876


In [ ]:
k_point = 0
band = 26
whichspin = 0

height = 7.
zero_phase = -.4
zoom_factor = 3
cells = [2, 1]

plot = structure_plot(structure, supercell = (cells[0], cells[1], 1), dimensions = 2)
plot.savefig(folder_name + "struc_2D.svg")
plt.show()
plot = structure_plot(structure, supercell = (cells[0], cells[1], 1), dimensions = 2, atom_size = .1, bond_size = .5)
plot.savefig(folder_name + "struc_2D_light.svg")
plt.show()

for band in range(luco_index - 10, luco_index + 9):
    orbital_plots(wavecar_data, k_point = k_point, band = band, height = height, cells = cells, zero_phase = zero_phase, zoom_factor = zoom_factor)

In [515]:
k_point = 0
band = 263
whichspin = 0

height = 7.
zero_phase = -.4
zoom_factor = 3
cells = [2, 1]

Fourier_cube = get_Fourier_cube(wavecar_data = wavecar_data, k_point = k_point, band = band, whichspin = 0)

Orbital number 263 with spin 0 (spin up) at k-point 0 and energy -2.836109307544641 eV. This orbital has pi symmetry


In [ ]:
plt.plot(np.sum(np.abs(Fourier_cube[:8, :20, :20]), axis = (1, 2)))
plt.xticks(np.arange(0, 8, 1))
plt.grid("both")
plt.show()

Tight-binding

In [ ]:
N_GNR = 5
GNR_orientation = "A" # "A" for armchair or "Z" for zigzag
defect_row = 2
defect_type = "Na" # Swap to something else for primitive cell calculations
supercell_length = 12

if defect_type != "N" and defect_type != "B": folder_name = "C:/DFT/" + str(N_GNR) + "-" + GNR_orientation + "GNR/" + str(N_GNR) + "-" + GNR_orientation + "GNR_primitive/" # Primitive GNRS
else: folder_name = "C:/DFT/" + str(N_GNR) + "-" + GNR_orientation + "GNR/" + str(N_GNR) + "-" + GNR_orientation + "GNR_" + str(supercell_length) + "_" + str(defect_type) + "_row" + str(defect_row) + "/"

structure = Structure.from_file(folder_name + "CONTCAR")
structure_plot(structure)

In [ ]:
H_nn = np.zeros((10, 10), dtype = int)
xyz = np.empty((0, 2), dtype = "float")
for i in range(len(structure.sites)):
    if structure.sites[i].species_string == "N" or structure.sites[i].species_string == "C": xyz = np.vstack([xyz, structure.sites[i].coords[:2]])

nn_dist = 1.6
for i in range(len(xyz)):
    for j in range(i + 1, len(xyz)):
        if np.linalg.norm(xyz[i] - xyz[j]) < nn_dist:
            H_nn[i, j] = 1

H_nn += H_nn.T
plt.imshow(H_nn)
plt.show()
eigenvalues, eigenvectors = np.linalg.eig(H_nn)
plt.plot(np.sort(eigenvalues))
plt.show()

Create STL files

In [235]:
structure = Structure.from_file(file_name)
structure.make_supercell((1, 1, 1))
lat_vec = structure.lattice
center = np.asarray(lat_vec.abc) * 0.5
structure.make_supercell((5, 1, 1))

spheres, cylinders = create_3D_model(structure, atom_size = 0.25, bond_size = 1, max_bondlength = 1.6, resolution = 30)
#o3d.visualization.draw_geometries(cylinders + spheres, lookat = center, zoom = 0.4, up = [0, 1, 0])

combined_mesh = spheres[0]
for i in range(1, len(spheres)):
    combined_mesh += spheres[i]
for i in range(len(cylinders)):
    combined_mesh += cylinders[i]

print(f"Combined mesh has {len(combined_mesh.vertices)} vertices and {len(combined_mesh.triangles)} triangles.")

# Visualize the combined mesh (optional)
#o3d.visualization.draw_geometries([combined_mesh])

combined_mesh.compute_vertex_normals()
o3d.io.write_triangle_mesh(folder_name + "structure.stl", combined_mesh)

Combined mesh has 1254240 vertices and 2505600 triangles.


True

In [283]:
N = 3

xlist = np.zeros(N * 2, dtype = float)
ylist = np.zeros(N * 2, dtype = float)
atomlist = np.zeros(N * 2, dtype = str)
CC_length = 1.42
CH_length = 1.09
sqrt3_2 = np.sqrt(3) / 2
row_to_row = .5 * CC_length * sqrt3_2 # Row-to-row spacing in the y direction

for i in range(0, N * 2, 2):
    xlist[i] = CC_length * (.25 * np.mod(i, 4) + .5) # Atom to the right side of the x axis
    xlist[i + 1] = -CC_length * (.25 * np.mod(i, 4) + .5) # Atom to the left side of the x axis
    ylist[i] = ylist[i + 1] = i * row_to_row
    atomlist[i] = "C"
    atomlist[i + 1] = "C"

# Pad hydrogen atoms to the bottom and top of the GNR
xlist = np.append(xlist, [.5 * CC_length + .5 * CH_length, -.5 * CC_length - .5 * CH_length,
                        -CC_length * (.25 * np.mod((N + 1) * 2, 4) + .5) + CH_length * (.5 - np.mod(N, 2)), CC_length * (.25 * np.mod((N + 1) * 2, 4) + .5) - CH_length * (.5 - np.mod(N, 2))])
ylist = np.append(ylist, [-CH_length * np.sqrt(3) / 2, -CH_length * np.sqrt(3) / 2,
                        np.max(ylist) + CH_length * sqrt3_2, np.max(ylist) + CH_length * sqrt3_2])
atomlist = np.append(atomlist, ["H", "H", "H", "H"])
ylist -= np.mean(ylist) # Shift the y coordinates so that the structure is centered around the origin

latvec_x = 3 * CC_length # Simply the unit cell length
latvec_y = (N + 1) * 4 * row_to_row # The GNR hard wall boundary conditions dictate that the transverse component of the wavefunctions have nodal planes on atomic row 0 and atomic row N + 1
# Thus, using this width ensures that the unit cell exactly fits the wavelength of the transverse waves and integer multiples of it
# This prevents having to represent the transverse nodal plane structure with lots of different waves, also known as Fourier leakage
latvec_z = 10 # 1 nm distance in the height direction should be sufficient to limit interactions between periodic images

xlist_shifted = xlist + .5 * latvec_x # Move the atoms from being centered around the origin to being centered around the center of the unit cell
ylist_shifted = ylist + .5 * latvec_y

structure = Structure(lattice = Lattice.from_parameters(latvec_x, latvec_y, latvec_z, 90, 90, 90), species = atomlist,
                    coords = np.array([xlist_shifted, ylist_shifted, np.zeros_like(xlist) + .5 * latvec_z]).T, coords_are_cartesian = True)

structure.make_supercell((9, 1, 1))

structure_plot(structure, max_bondlength = 1.6)

Band unfolding

In [12]:
N_ribbon = 4
len_sc = 8
dopant = "N"
dopant_row = 2

folder_name_pc = "C:/DFT/" + str(N_ribbon) + "-AGNR/" + str(N_ribbon) + "-AGNR_primitive/"
file_name_pc = folder_name_pc + "CONTCAR"

structure_pc, lattice_pc, center_pc = structure_loader(file_name_pc)
n_bands_pc, n_kpoints_pc, n_ions, n_noH, n_orbitals, spin_polarized, orbital_list, pz_index, luco_index_pc, hoco_energy_pc, luco_energy_pc, midgap_energy_pc, band_gap_pc, procar_data_pc = procar_loader(folder_name_pc)
pc_latt = lattice_pc.matrix

folder_name_sc = "C:/DFT/" + str(N_ribbon) + "-AGNR/" + str(N_ribbon) + "-AGNR_" + str(len_sc) + "_" + dopant + "_row" + str(dopant_row) + "/"
file_name_sc = folder_name_sc + "CONTCAR"

structure_sc, lattice_sc, center_sc = structure_loader(file_name_sc)
n_bands_sc, n_kpoints_sc, n_ions, n_noH, n_orbitals, spin_polarized, orbital_list, pz_index, luco_index_sc, hoco_energy_sc, luco_energy_sc, midgap_energy_sc, band_gap_sc, procar_data_sc = procar_loader(folder_name_sc)
sc_latt = lattice_sc.matrix

sc_matrix = np.diag([lattice_sc.a / lattice_pc.a, lattice_sc.b / lattice_pc.b, lattice_sc.c / lattice_pc.c])
print(sc_matrix)

Number of k points: 40; number of bands: 48; number of atoms: 12; number of hydrogen atoms: 4; number of orbitals: 9
Identity of orbitals:  ['s', 'py', 'pz', 'px', 'dxy', 'dyz', 'dz2', 'dxz', 'x2-y2']
The VB maximum is at -3.91017063 eV and the CB minimum is at -1.45477009 eV, giving a band gap of 2.4554005400000003 eV
Spin polarized calculation: False
Number of k points: 5; number of bands: 368; number of atoms: 96; number of hydrogen atoms: 32; number of orbitals: 9
Identity of orbitals:  ['s', 'py', 'pz', 'px', 'dxy', 'dyz', 'dz2', 'dxz', 'x2-y2']
The VB maximum is at -2.04061611 eV and the CB minimum is at -1.5053 eV, giving a band gap of 0.5353161100000001 eV
Spin polarized calculation: True
[[8. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [ ]:
atoms_pc = AseAtomsAdaptor.get_atoms(structure_pc)
pc_opts = unfold.get_symmetry_dataset(atoms_pc)
atoms_sc = AseAtomsAdaptor.get_atoms(structure_sc)
sc_opts = unfold.get_symmetry_dataset(atoms_sc)

unfold_obj = unfold.Unfold(sc_matrix, folder_name_pc + "WAVECAR")

In [394]:
unfold_obj = unfold.UnfoldKSet(sc_matrix, k_points_pc, pc_latt, pc_opts.rotations, sc_opts.rotations)

In [30]:
from vaspwfc import vaspwfc
from unfold import removeDuplicateKpoints, find_K_from_k, save2VaspKPOINTS, unfold

In [ ]:
kpath = procar_data_pc.kpoints

K_in_sup = []
for kk in kpath:
    kg, g = find_K_from_k(kk, sc_matrix)
    K_in_sup.append(kg)

print(np.array(K_in_sup))

reducedK, kid = removeDuplicateKpoints(K_in_sup, return_map = True)
reducedK
save2VaspKPOINTS(reducedK)

In [ ]:
wavesuper = unfold(M = sc_matrix, wavecar = folder_name_sc + "WAVECAR")

from unfold import EBS_cmaps
sw = wavesuper.spectral_weight(K_in_sup)
e0, sf = wavesuper.spectral_function(nedos = 4000)
# or show the effective band structure with colormap
EBS_cmaps(kpath, pc_latt, e0, sf, nseg = 39, eref = -1.01, show = False, ylim = (-7, 7))

In [5]:
folder_name = "C:/Research/DFTfiles/"
file_name = "WAVECAR"

n_bands, n_kpoints, spin_polarized, luco_index, hoco_energy, luco_energy, midgap_energy, band_gap, wavecar_data = wavecar_loader(folder_name)

Number of k points: 40; number of bands: 64
The VB maximum is at -3.170482268382681 eV and the CB minimum is at -2.0516558456921663 eV, giving a band gap of 1.118826422690515 eV
Spin polarized calculation: False
